In [1]:
import sys
import os

notebook_path = os.getcwd() 
parent_dir = os.path.dirname(notebook_path)
project_root = os.path.dirname(parent_dir)
if project_root not in sys.path:
    sys.path.insert(0, project_root)

In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import matplotlib.pyplot as plt
from copy import deepcopy

In [3]:
import state_NN_models
import Filters
import utils
import Systems
from utils import losses, trainer, utils
from torch.utils.data import TensorDataset, DataLoader, random_split
from state_NN_models.StateBayesianKalmanNet import StateBayesianKalmanNet
from state_NN_models.StateKalmanNet import StateKalmanNet
from state_NN_models.StateKalmanNetWithKnownR import StateKalmanNetWithKnownR

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Používané zařízení: {device}")

Používané zařízení: cuda


In [ ]:
import math
import torch

state_dim_2d = 2
obs_dim_2d = 2

# ==============================================================
# 1. SKUTEČNÝ SYSTÉM (2D Pozice - Random Walk)
# ==============================================================
# Matice identity. Vlastní čísla jsou přesně 1.0. 
# Stav nereprezentuje rychlost, jen X a Y pozici, která se posouvá pomocí šumu Q.
F_true_2d = torch.tensor([[1.0, 0.0], 
                          [0.0, 1.0]])
print("F_true_2d (Skutečná dynamika):\n", F_true_2d)

H_true_2d = torch.eye(obs_dim_2d)

Q_true_2d = torch.eye(state_dim_2d) * 0.1 # Šum procesu pohání trajektorii
R_true_2d = torch.eye(obs_dim_2d) * 2.0   # Hodně zašuměná měření

Ex0_true_2d = torch.tensor([[1.0], [0.0]])
P0_true_2d = torch.eye(state_dim_2d) * 1.5

# ==============================================================
# 2. MODEL SYSTÉMU (Zavedení čisté rotace - Mismatch)
# ==============================================================
delta_alpha_deg = 10.0
delta_alpha_rad = delta_alpha_deg * math.pi / 180.0

# Vytvoření 2D rotační matice R(Δα)
rotation_matrix = torch.tensor([
    [math.cos(delta_alpha_rad), -math.sin(delta_alpha_rad)],
    [math.sin(delta_alpha_rad),  math.cos(delta_alpha_rad)]
])

# Model předpokládá, že se poloha v každém kroku pootočí o 10 stupňů
# Vlastní čísla čisté rotační matice mají velikost přesně 1.0. EXPLOSE SE NEKONÁ!
F_model_2d = torch.matmul(rotation_matrix, F_true_2d)
print("\nF_model_2d (S chybou rotace o 10°):\n", F_model_2d)

H_model_2d = H_true_2d
Q_model_2d = Q_true_2d
R_model_2d = R_true_2d

Ex0_model_2d = Ex0_true_2d
P0_model_2d = P0_true_2d

# Incializace sys_true a sys_model zůstává stejná...
# ==============================================================
# 3. INICIALIZACE OBJEKTŮ SYSTÉMU
# ==============================================================
print("\nInicializuji 2D Linear_Canonical systém (Realita vs. Model)...")
sys_true = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_true_2d, P0=P0_true_2d,
    Q=Q_true_2d, R=R_true_2d,
    F=F_true_2d, H=H_true_2d,
    device=device
)

sys_model = Systems.DynamicSystem(
    state_dim=state_dim_2d, obs_dim=obs_dim_2d,
    Ex0=Ex0_model_2d, P0=P0_model_2d,
    Q=Q_model_2d, R=R_model_2d,
    F=F_model_2d, H=H_model_2d,
    device=device
)

print("... 2D systémy úspěšně inicializovány s modelovým mismatchem.")

F_true_2d (Skutečná dynamika):
 tensor([[1., 0.],
        [0., 1.]])

F_model_2d (S chybou rotace o 10°):
 tensor([[ 0.9848, -0.1736],
        [ 0.1736,  0.9848]])

Inicializuji 2D Linear_Canonical systém (Realita vs. Model)...
... 2D systémy úspěšně inicializovány s modelovým mismatchem.


In [ ]:
TRAIN_SEQ_LEN = 10      # Krátké sekvence pro stabilní trénink (TBPTT)
VALID_SEQ_LEN = 100      # Stejná délka pro konzistentní validaci
TEST_SEQ_LEN = 1000      # Dlouhé sekvence pro testování generalizace

NUM_TRAIN_TRAJ = 500   # Hodně trénovacích příkladů
NUM_VALID_TRAJ = 100    # Dostatek pro spolehlivou validaci
NUM_TEST_TRAJ = 100     # Pro robustní vyhodnocení

BATCH_SIZE = 8         # Dobrý kompromis

x_train, y_train = utils.generate_data(sys_true, num_trajectories=NUM_TRAIN_TRAJ, seq_len=TRAIN_SEQ_LEN)
x_val, y_val = utils.generate_data(sys_true, num_trajectories=NUM_VALID_TRAJ, seq_len=VALID_SEQ_LEN)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)

train_dataset = TensorDataset(x_train, y_train)
val_dataset = TensorDataset(x_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy

model_config = {
    "hidden_size_multiplier": 10,
    "output_layer_multiplier": 4,
    "num_gru_layers": 1,
    "init_min_dropout": 0.5,
    "init_max_dropout": 0.8
}

train_config = {
    "total_train_iter": 1400,
    "learning_rate": 1e-4,
    "clip_grad": 10.0,
    "J_samples": 5,
    "validation_period": 15,
    "logging_period": 10,
    "warmup_iterations": 0, # Trénuj prvních 100 iterací jen na MSE
    "store_model_based_on_hybrid_score": True,
    "calibration_parameter": 0.15
}

# =================================================================================
# KROK 3: SPUŠTĚNÍ JEDNOHO TRÉNINKOVÉHO BĚHU NEBO NAČTENÍ HOTOVÉHO MODELU
# =================================================================================

print("="*80)
print("Připravuji Bayesian KalmanNet (BKN)...")
print(f"Parametry modelu: {model_config}")
print(f"Parametry tréninku: {train_config}")
print("="*80)

# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

# Definice cesty ke složce a souboru
save_dir = "LinearSystemWeightsMismatch"
os.makedirs(save_dir, exist_ok=True)
weights_path = os.path.join(save_dir, "best_bkn_model_weights_mismatch.pth")

# Vytvoření modelu (potřebujeme ho inicializovat v obou případech)
state_bkn_knet = StateBayesianKalmanNet(
    sys_model,
    device=device,
    weight_init=False,
    **model_config
).to(device)

# Podmínka pro načtení nebo trénink
if os.path.exists(weights_path):
    print(f"\nINFO: Uložené váhy nalezeny v '{weights_path}'.")
    print("Přeskakuji trénink a načítám hotový model...")
    
    # Načtení vah
    state_bkn_knet.load_state_dict(torch.load(weights_path, map_location=device))
    trained_model = state_bkn_knet
    
else:
    print("\nINFO: Váhy nebyly nalezeny. Spouštím plnohodnotný tréninkový běh...")
    
    # Spuštění tréninku
    results = trainer.training_session_trajectory_with_gaussian_nll_training_fcn(
        model=state_bkn_knet,
        train_loader=train_loader,
        val_loader=val_loader,
        device=device,
        **train_config
    )

    # `run_training_session` automaticky načte nejlepší model zpět,
    # takže `state_bkn_knet` nyní obsahuje váhy nejlepšího modelu.
    trained_model = results['final_model']
    
    # Uložení vah pro příští použití
    torch.save(trained_model.state_dict(), weights_path)
    print(f"\nINFO: Váhy modelu byly úspěšně uloženy do: {weights_path}")

    # Výpis metrik pouze pokud jsme trénovali
    print("\n" + "="*80)
    print("TRÉNINK DOKONČEN - FINÁLNÍ VÝSLEDKY Z NEJLEPŠÍHO MODELU")
    print("="*80)
    print(f"Nejlepší model byl nalezen v iteraci: {results['best_iter']}")
    print(f"Nejlepší dosažený validační ANEES: {results['best_val_anees']:.4f}")
    print("--- Metriky odpovídající tomuto nejlepšímu modelu ---")
    print(f"  MSE na validační sadě:       {results['best_val_mse']:.4f}")
    print(f"  NLL na validační sadě:       {results['best_val_nll']:.4f}")
    print("="*80)

# Prepnutí do evaluačního módu na konci bloku (klíčové pro BKN kvůli Dropoutu)
trained_model.eval()
print("\nModel BKN je plně připraven v proměnné 'trained_model'.")

Připravuji Bayesian KalmanNet (BKN)...
Parametry modelu: {'hidden_size_multiplier': 10, 'output_layer_multiplier': 4, 'num_gru_layers': 1, 'init_min_dropout': 0.5, 'init_max_dropout': 0.8}
Parametry tréninku: {'total_train_iter': 1400, 'learning_rate': 0.0001, 'clip_grad': 10.0, 'J_samples': 5, 'validation_period': 15, 'logging_period': 5000, 'warmup_iterations': 200, 'store_model_based_on_hybrid_score': True, 'calibration_parameter': 0.15}
loaded with input normalization

INFO: Váhy nebyly nalezeny. Spouštím plnohodnotný tréninkový běh...

--- Validation at iteration 15 ---
  Average MSE: 5.8171, Average ANEES: 30.7367
  Hybrid Score: 10.1276 (Penalty: 28.7367 * 0.15)
  >>> New best HYBRID SCORE! (10.1276 < inf) Saving model. <<<
--------------------------------------------------

--- Validation at iteration 30 ---
  Average MSE: 2.6807, Average ANEES: 34.1254
  Hybrid Score: 7.4995 (Penalty: 32.1254 * 0.15)
  >>> New best HYBRID SCORE! (7.4995 < 10.1276) Saving model. <<<
-----------

KeyboardInterrupt: 

In [8]:

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import os
import random
import csv
from datetime import datetime
import pandas as pd
from copy import deepcopy
# Nastavení seedu pro reprodukovatelnost tohoto běhu
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

save_dir = "LinearSystemWeightsMismatch"
weights_path = os.path.join(save_dir, "best_knet_model_weights_mismatch.pth")

# Vytvoření cílové složky (pokud ještě neexistuje, nic se nestane)
os.makedirs(save_dir, exist_ok=True)


state_knetR = StateKalmanNet(sys_model, device=device, hidden_size_multiplier=10,returns_covariance=True).to(device)

if os.path.exists(weights_path):
    print(f"INFO: Uložené váhy nalezeny v '{weights_path}'.")
    print("Přeskakuji trénink a načítám hotový model...")
    
    # Načtení vah (map_location zajistí kompatibilitu, pokud bys např. trénoval na GPU a načítal na CPU)
    state_knetR.load_state_dict(torch.load(weights_path, map_location=device))
    
else:
    print("INFO: Váhy nebyly nalezeny. Spouštím trénink modelu...")
    
    # Spuštění tréninku
    trainer.train_state_KalmanNet(
        model=state_knetR, 
        train_loader=train_loader, 
        val_loader=val_loader, 
        device=device, 
        epochs=200, 
        lr=1e-4,
        early_stopping_patience=30,
        weight_decay=1e-3
    )
    
    # Po úspěšném tréninku váhy rovnou uložíme pro příští spuštění
    torch.save(state_knetR.state_dict(), weights_path)
    print(f"Trénink dokončen. Váhy byly bezpečně uloženy do '{weights_path}'.")

# Prepnutí do evaluačního módu (dobrá praxe před testováním)
state_knetR.eval()

DEBUG: Layer 'output_final_linear.0' initialized near zero (Start K=0).
INFO: Váhy nebyly nalezeny. Spouštím trénink modelu...
New best model saved! Epoch [1/200], Train Loss: 7.296412, Val Loss: 13.766607
New best model saved! Epoch [2/200], Train Loss: 6.744071, Val Loss: 12.879645
New best model saved! Epoch [3/200], Train Loss: 6.329579, Val Loss: 12.084398
New best model saved! Epoch [4/200], Train Loss: 5.955346, Val Loss: 11.306313
Epoch [5/200] | Train Loss: 5.5379 (Pos: 0.00, Vel: 0.00) | Val Loss: 10.5480 | Val MSE: 10.55
Epoch [4/200], Train Loss: 5.955346, Val Loss: 11.306313, Avg Cov Trace: 0.214491
New best model saved! Epoch [5/200], Train Loss: 5.537877, Val Loss: 10.548003
New best model saved! Epoch [6/200], Train Loss: 5.171049, Val Loss: 9.745652
New best model saved! Epoch [7/200], Train Loss: 4.727957, Val Loss: 8.880860
New best model saved! Epoch [8/200], Train Loss: 4.263947, Val Loss: 7.921686
New best model saved! Epoch [9/200], Train Loss: 3.731508, Val Loss

RuntimeError: stack expects each tensor to be equal size, but got [52, 2, 2] at entry 0 and [51, 2, 2] at entry 89

In [11]:
import torch
import torch.nn.functional as F
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import Filters

# ==============================================================================
# 0. PŘEDPOKLADY - ZDE PŘIŘAĎTE VAŠE NATRÉNOVANÉ MODELY
# ==============================================================================
# POZNÁMKA: Sítě trained_model_bkn a trained_model_knetR musí být inicializovány 
# s objektem `sys_model` (chybná dynamika) před spuštěním tohoto skriptu!
try:
    trained_model_bkn = trained_model
    trained_model_knetR = state_knetR
    print("INFO: Všechny natrénované modely nalezeny a přiřazeny.")
except NameError:
    print("VAROVÁNÍ: Některé z proměnných `trained_model`, nebo `state_knetR` nebyly nalezeny.")
    print("         Ujistěte se, že jste nejprve úspěšně dokončili trénink všech modelů.")

# ==============================================================================
# 1. KONFIGURACE TESTU
# ==============================================================================
TEST_SEQ_LEN = 1000
NUM_TEST_TRAJ = 10
J_SAMPLES_TEST = 50

# ==============================================================================
# 2. PŘÍPRAVA DAT (Generováno z ideálního / skutečného světa)
# ==============================================================================
print(f"\nGeneruji {NUM_TEST_TRAJ} testovacích trajektorií o délce {TEST_SEQ_LEN}...")
# Data generujeme VŽDY pomocí sys_true (skutečná fyzika)
x_test, y_test = utils.generate_data(sys_true, num_trajectories=NUM_TEST_TRAJ, seq_len=TEST_SEQ_LEN)
test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)
print("Generování dat dokončeno.")

# ==============================================================================
# 3. INICIALIZACE FILTRŮ (TEST NA NEPŘESNÉM MODELU)
# ==============================================================================
# Všechny klasické filtry nyní dostanou chybný sys_model
kf_mismatched = Filters.KalmanFilter(sys_model)
ekf_mismatched = Filters.ExtendedKalmanFilter(sys_model)
ukf_mismatched = Filters.UnscentedKalmanFilter(sys_model)
akf_mdm_mismatched = Filters.AdaptiveKalmanFilter_default(sys_model, mdm_L=3, mdm_version=2, lambda_rls=1.0, init_sigma_rls_value=0.1) 
akf_mehra_mismatched = Filters.AdaptiveKalmanFilter_mehra_offline(sys_model, num_iterations=5)

# Ponecháme si jeden čistě ideální KF jako referenci (Spodní mez chyby)
kf_ideal = Filters.KalmanFilter(sys_true)

print("Klasické filtry inicializovány s NEPŘESNÝM modelem (sys_model).")
print("Ideální KF inicializován s PŘESNÝM modelem (sys_true) jako benchmark.")

# ==============================================================================
# 4. VYHODNOCOVACÍ SMYČKA
# ==============================================================================
all_x_true_cpu = []
all_x_hat_bkn_cpu, all_P_hat_bkn_cpu = [], []
all_x_hat_knetR_cpu, all_P_hat_knetR_cpu = [], []
all_x_hat_kf_m_cpu, all_P_hat_kf_m_cpu = [], []
all_x_hat_ekf_m_cpu, all_P_hat_ekf_m_cpu = [], []
all_x_hat_ukf_m_cpu, all_P_hat_ukf_m_cpu = [], []
all_x_hat_akf_mdm_m_cpu, all_P_hat_akf_mdm_m_cpu = [], []
all_x_hat_akf_mehra_m_cpu = [] 
all_x_hat_kf_ideal_cpu, all_P_hat_kf_ideal_cpu = [], []

print(f"\nVyhodnocuji modely na {NUM_TEST_TRAJ} testovacích trajektoriích...")

trained_model_bkn.eval() 
trained_model_knetR.eval()

with torch.no_grad():
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        
        # --- A. Bayesian KalmanNet (Trajectory-wise) ---
        ensemble_trajectories = []
        for j in range(J_SAMPLES_TEST):
            trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
            current_x_hats = []
            for t in range(1, TEST_SEQ_LEN):
                x_filtered_t, _ = trained_model_bkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
                current_x_hats.append(x_filtered_t)
            ensemble_trajectories.append(torch.cat(current_x_hats, dim=0))
        ensemble = torch.stack(ensemble_trajectories, dim=0)
        predictions_bkn = ensemble.mean(dim=0)
        diff = ensemble - predictions_bkn.unsqueeze(0)
        covariances_bkn = (diff.unsqueeze(-1) @ diff.unsqueeze(-2)).mean(dim=0)
        full_x_hat_bkn = torch.cat([initial_state, predictions_bkn], dim=0)
        # AI startuje z chybného P0
        full_P_hat_bkn = torch.cat([sys_model.P0.unsqueeze(0), covariances_bkn], dim=0)
        print(f"Trajektorie {i + 1}/{NUM_TEST_TRAJ}: Bayesian KNet (BKN) dokončen.")

        # --- B. StateKalmanNetWithKnownR ---
        trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        knetR_preds_x, knetR_preds_P = [], []
        for t in range(1, TEST_SEQ_LEN):
            x_filtered_t, P_filtered_t = trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
            knetR_preds_x.append(x_filtered_t)
            knetR_preds_P.append(P_filtered_t)
        full_x_hat_knetR = torch.cat([initial_state, torch.cat(knetR_preds_x, dim=0)], dim=0)
        
        processed_P_list = []
        for p_tensor in knetR_preds_P:
            while p_tensor.dim() < 2: p_tensor = p_tensor.unsqueeze(-1)
            if p_tensor.dim() > 2 and p_tensor.shape[0] == 1: p_tensor = p_tensor.squeeze(0)
            processed_P_list.append(p_tensor)

        P_sequence_knetR = torch.stack(processed_P_list, dim=0)
        P0_for_cat = sys_model.P0.clone()
        while P0_for_cat.dim() < P_sequence_knetR.dim(): P0_for_cat = P0_for_cat.unsqueeze(0)
        full_P_hat_knetR = torch.cat([P0_for_cat, P_sequence_knetR], dim=0)
        print(f"Trajektorie {i + 1}/{NUM_TEST_TRAJ}: StateKalmanNetWithKnownR (KNetR) dokončen.")

        # --- C. Klasické a adaptivní filtry (MISMATCHED) ---
        kf_m_res = kf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_kf_m = kf_m_res['x_filtered']
        full_P_hat_kf_m = kf_m_res['P_filtered']
        
        ekf_m_res = ekf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ekf_m = ekf_m_res['x_filtered']
        full_P_hat_ekf_m = ekf_m_res['P_filtered']
        
        ukf_m_res = ukf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        full_x_hat_ukf_m = ukf_m_res['x_filtered']
        full_P_hat_ukf_m = ukf_m_res['P_filtered']

        afk_mdm_m_res,_,_ = akf_mdm_mismatched.process_sequence_adaptively(y_test_seq_gpu)
        full_x_hat_akf_mdm_m = afk_mdm_m_res['x_filtered']
        full_P_hat_akf_mdm_m = afk_mdm_m_res['P_filtered'] 

        akf_mehra_m_res = akf_mehra_mismatched.process_sequence(y_test_seq_gpu)
        full_x_hat_akf_mehra_m = akf_mehra_m_res['x_filtered']   
        print(f"Trajektorie {i + 1}/{NUM_TEST_TRAJ}: Mismatched model-based filtry dokončeny.")

        # --- D. Benchmark (IDEAL) ---
        kf_i_res = kf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        full_x_hat_kf_ideal = kf_i_res['x_filtered']
        full_P_hat_kf_ideal = kf_i_res['P_filtered']
        print(f"Trajektorie {i + 1}/{NUM_TEST_TRAJ}: KF (Ideální Benchmark) dokončen.")
        
        # Ukládání na CPU
        all_x_true_cpu.append(x_true_seq_gpu.cpu())
        all_x_hat_bkn_cpu.append(full_x_hat_bkn.cpu()); all_P_hat_bkn_cpu.append(full_P_hat_bkn.cpu())
        all_x_hat_knetR_cpu.append(full_x_hat_knetR.cpu()); all_P_hat_knetR_cpu.append(full_P_hat_knetR.cpu())
        
        all_x_hat_kf_m_cpu.append(full_x_hat_kf_m.cpu()); all_P_hat_kf_m_cpu.append(full_P_hat_kf_m.cpu())
        all_x_hat_ekf_m_cpu.append(full_x_hat_ekf_m.cpu()); all_P_hat_ekf_m_cpu.append(full_P_hat_ekf_m.cpu())
        all_x_hat_ukf_m_cpu.append(full_x_hat_ukf_m.cpu()); all_P_hat_ukf_m_cpu.append(full_P_hat_ukf_m.cpu())
        all_x_hat_akf_mdm_m_cpu.append(full_x_hat_akf_mdm_m.cpu()); all_P_hat_akf_mdm_m_cpu.append(full_P_hat_akf_mdm_m.cpu())
        all_x_hat_akf_mehra_m_cpu.append(full_x_hat_akf_mehra_m.cpu())
        
        all_x_hat_kf_ideal_cpu.append(full_x_hat_kf_ideal.cpu()); all_P_hat_kf_ideal_cpu.append(full_P_hat_kf_ideal.cpu())

        print(f"Dokončena trajektorie {i + 1}/{NUM_TEST_TRAJ}...")

# ==============================================================================
# 5. FINÁLNÍ VÝPOČET A VÝPIS METRIK
# ==============================================================================
mse_bkn, anees_bkn = [], []; mse_knetR, anees_knetR = [], []
mse_kf_m, anees_kf_m = [], []
mse_ekf_m, anees_ekf_m = [], []
mse_ukf_m, anees_ukf_m = [], []
mse_akf_mdm_m, anees_akf_mdm_m = [], []
mse_akf_mehra_m = [] 
mse_kf_ideal, anees_kf_ideal = [], []

print("\nPočítám finální metriky pro jednotlivé trajektorie...")

with torch.no_grad():
    for i in range(NUM_TEST_TRAJ):
        x_true = all_x_true_cpu[i]
        
        def get_metrics(x_hat, P_hat):
            mse = F.mse_loss(x_hat[1:], x_true[1:]).item()
            anees = utils.calculate_anees_vectorized(x_true.unsqueeze(0), x_hat.unsqueeze(0), P_hat.unsqueeze(0))
            return mse, anees

        # Data-driven
        mse, anees = get_metrics(all_x_hat_bkn_cpu[i], all_P_hat_bkn_cpu[i]); mse_bkn.append(mse); anees_bkn.append(anees)
        mse, anees = get_metrics(all_x_hat_knetR_cpu[i], all_P_hat_knetR_cpu[i]); mse_knetR.append(mse); anees_knetR.append(anees)
        
        # Mismatched model-based
        mse, anees = get_metrics(all_x_hat_kf_m_cpu[i], all_P_hat_kf_m_cpu[i]); mse_kf_m.append(mse); anees_kf_m.append(anees)
        mse, anees = get_metrics(all_x_hat_ekf_m_cpu[i], all_P_hat_ekf_m_cpu[i]); mse_ekf_m.append(mse); anees_ekf_m.append(anees)
        mse, anees = get_metrics(all_x_hat_ukf_m_cpu[i], all_P_hat_ukf_m_cpu[i]); mse_ukf_m.append(mse); anees_ukf_m.append(anees)
        mse, anees = get_metrics(all_x_hat_akf_mdm_m_cpu[i], all_P_hat_akf_mdm_m_cpu[i]); mse_akf_mdm_m.append(mse); anees_akf_mdm_m.append(anees)
        mse_akf_mehra_m.append(F.mse_loss(all_x_hat_akf_mehra_m_cpu[i][1:], x_true[1:]).item())
        
        # Benchmark ideal
        mse, anees = get_metrics(all_x_hat_kf_ideal_cpu[i], all_P_hat_kf_ideal_cpu[i]); mse_kf_ideal.append(mse); anees_kf_ideal.append(anees)

def avg(metric_list): return np.mean([m for m in metric_list if not np.isnan(m)])
state_dim_for_nees = all_x_true_cpu[0].shape[1]

# --- Finální výpis tabulky ---
print("\n" + "="*85)
print(f"FINÁLNÍ VÝSLEDKY: MODEL MISMATCH SCÉNÁŘ (průměr přes {NUM_TEST_TRAJ} běhů)")
print("Filtrům byla dodána chybná rotace F_model a chybný šum Q_model.")
print("="*85)
print(f"{'Model':<35} | {'Průměrné MSE':<20} | {'Průměrný ANEES':<20}")
print("-" * 85)
print(f"{'--- AI Filtry (Trénováno na chybě) ---':<35} | {'(nižší je lepší)':<20} | {'(bližší ' + str(float(state_dim_for_nees)) + ' je lepší)':<20}")
print(f"{'Bayesian KNet (BKN)':<35} | {avg(mse_bkn):<20.4f} | {avg(anees_bkn):<20.4f}")
print(f"{'KNet with Known R (KNetR)':<35} | {avg(mse_knetR):<20.4f} | {avg(anees_knetR):<20.4f}")
print("-" * 85)
print(f"{'--- Klasické Filtry (Chybný model) ---':<35} | {'':<20} | {'':<20}")
print(f"{'KF (Nepřesný model)':<35} | {avg(mse_kf_m):<20.4f} | {avg(anees_kf_m):<20.4f}")
print(f"{'EKF (Nepřesný model)':<35} | {avg(mse_ekf_m):<20.4f} | {avg(anees_ekf_m):<20.4f}")
print(f"{'UKF (Nepřesný model)':<35} | {avg(mse_ukf_m):<20.4f} | {avg(anees_ukf_m):<20.4f}")
print(f"{'AKF-MDM (Nepřesný model)':<35} | {avg(mse_akf_mdm_m):<20.4f} | {avg(anees_akf_mdm_m):<20.4f}")
print(f"{'AKF-Mehra (Nepřesný model)':<35} | {avg(mse_akf_mehra_m):<20.4f} | {'N/A':<20}")
print("-" * 85)
print(f"{'--- BENCHMARK (Fyzikální limit) ---':<35} | {'':<20} | {'':<20}")
print(f"{'KF (Ideální model - OPTIMUM)':<35} | {avg(mse_kf_ideal):<20.4f} | {avg(anees_kf_ideal):<20.4f}")
print("="*85)

VAROVÁNÍ: Některé z proměnných `trained_model`, nebo `state_knetR` nebyly nalezeny.
         Ujistěte se, že jste nejprve úspěšně dokončili trénink všech modelů.

Generuji 10 testovacích trajektorií o délce 1000...
Generování dat dokončeno.
Klasické filtry inicializovány s NEPŘESNÝM modelem (sys_model).
Ideální KF inicializován s PŘESNÝM modelem (sys_true) jako benchmark.

Vyhodnocuji modely na 10 testovacích trajektoriích...


NameError: name 'trained_model_bkn' is not defined

# Kalman Gain comparison

In [ ]:
import matplotlib.pyplot as plt
import torch
import numpy as np

# ==============================================================================
# ANALÝZA VÝVOJE KALMANOVA ZISKU A POROVNÁNÍ VŮČI OPTIMU (FROBENIOVA NORMA)
# SCÉNÁŘ: MODEL MISMATCH
# ==============================================================================
num_plot_traj = min(3, NUM_TEST_TRAJ)

print("Předpočítávám výsledky filtrů a extrahuji Kalmanovy zisky pro vykreslení...")

# Seznamy pro uložení historie chyb (Frobeniova norma)
frob_errors_kf_m = []
frob_errors_bkn = []
frob_errors_knetR = []
frob_errors_mdm = []
frob_errors_mehra = []

trained_model_bkn.eval()
trained_model_knetR.eval()

with torch.no_grad():
    # Procházíme testovací trajektorie
    for i, (x_true_seq_batch, y_test_seq_batch) in enumerate(test_loader):
        if i >= num_plot_traj:
            break
            
        y_test_seq_gpu = y_test_seq_batch.squeeze(0).to(device)
        x_true_seq_gpu = x_true_seq_batch.squeeze(0).to(device)
        initial_state = x_true_seq_gpu[0, :].unsqueeze(0)
        seq_len = y_test_seq_gpu.shape[0]

        # --- 1. Ideální KF (Benchmark - Absolutní Optimum) ---
        res_ideal = kf_ideal.process_sequence(y_test_seq_gpu, Ex0=sys_true.Ex0, P0=sys_true.P0)
        K_kf_ideal = res_ideal['Kalman_gain'].cpu()  # Očekávaný tvar: [seq_len, state_dim, obs_dim]

        # --- 2. Nepřesný KF (Referenční spodní hranice inteligence) ---
        res_kf_m = kf_mismatched.process_sequence(y_test_seq_gpu, Ex0=sys_model.Ex0, P0=sys_model.P0)
        K_kf_m = res_kf_m['Kalman_gain'].cpu()
        
        # --- 3. AKF MDM (Nepřesný model) ---
        res_mdm,_,_ = akf_mdm_mismatched.process_sequence_adaptively(y_test_seq_gpu)
        K_mdm = res_mdm['Kalman_gain'].cpu()
        
        # --- 4. AKF Mehra (Nepřesný model) ---
        res_mehra = akf_mehra_mismatched.process_sequence(y_test_seq_gpu)
        K_mehra = res_mehra['Kalman_gain'].cpu()
        
        # Ošetření: Pokud Mehra vrací konstantní zisk pro celou trajektorii, expandujeme ho
        if K_mehra.dim() == 2:
            K_mehra = K_mehra.unsqueeze(0).expand(seq_len, -1, -1)
        elif K_mehra.shape[0] < seq_len:
            pad_len = seq_len - K_mehra.shape[0]
            K_mehra = torch.cat([K_kf_ideal[:pad_len], K_mehra], dim=0)

        # --- 5. Bayesian KalmanNet (Zprůměrovaný zisk přes J vzorků) ---
        K_bkn_ensemble = []
        for j in range(J_SAMPLES_TEST):
            trained_model_bkn.reset(batch_size=1, initial_state=initial_state)
            for t in range(1, seq_len):
                trained_model_bkn.step(y_test_seq_gpu[t, :].unsqueeze(0))
            
            K_history = trained_model_bkn.get_kalman_gain_history().squeeze(1).cpu()
            # AI aktualizuje zisk až od t=1, nultý krok doplníme z nepřesného P0 (získaného přes K_kf_m)
            K_history_full = torch.cat([K_kf_m[0].unsqueeze(0), K_history], dim=0)
            K_bkn_ensemble.append(K_history_full)
        
        K_bkn = torch.stack(K_bkn_ensemble, dim=0).mean(dim=0)

        # --- 6. KalmanNet s Known R (KNetR) ---
        trained_model_knetR.reset(batch_size=1, initial_state=initial_state)
        for t in range(1, seq_len):
            trained_model_knetR.step(y_test_seq_gpu[t, :].unsqueeze(0))
            
        K_history_knetR = trained_model_knetR.get_kalman_gain_history().squeeze(1).cpu()
        K_knetR = torch.cat([K_kf_m[0].unsqueeze(0), K_history_knetR], dim=0)

        # --- VÝPOČET FROBENIOVY NORMY CHYBY ||K_model - K_KF_IDEAL||_F ---
        err_kf_m = torch.linalg.matrix_norm(K_kf_m - K_kf_ideal, ord='fro', dim=(1,2))
        err_bkn = torch.linalg.matrix_norm(K_bkn - K_kf_ideal, ord='fro', dim=(1,2))
        err_knetR = torch.linalg.matrix_norm(K_knetR - K_kf_ideal, ord='fro', dim=(1,2))
        err_mdm = torch.linalg.matrix_norm(K_mdm - K_kf_ideal, ord='fro', dim=(1,2))
        err_mehra = torch.linalg.matrix_norm(K_mehra - K_kf_ideal, ord='fro', dim=(1,2))
        
        frob_errors_kf_m.append(err_kf_m)
        frob_errors_bkn.append(err_bkn)
        frob_errors_knetR.append(err_knetR)
        frob_errors_mdm.append(err_mdm)
        frob_errors_mehra.append(err_mehra)

# ==============================================================================
# GRAFICKÁ A NUMERICKÁ PREZENTACE VÝSLEDKŮ
# ==============================================================================
print("\nGeneruji grafy konvergence Frobeniovy normy pro scénář Mismatch...")

fig, axes = plt.subplots(num_plot_traj, 1, figsize=(12, 5 * num_plot_traj), sharex=True)
if num_plot_traj == 1: axes = [axes]

steady_state_ratio = 0.2
idx_steady = int((1 - steady_state_ratio) * TEST_SEQ_LEN)

num_kf_m_ss, num_bkn_ss, num_knetR_ss, num_mdm_ss, num_mehra_ss = [], [], [], [], []

for i in range(num_plot_traj):
    ax = axes[i]
    
    # Referenční úroveň chyby, pokud by systém vůbec neadaptoval
    ax.plot(frob_errors_kf_m[i].numpy(), label='Statický KF (Chybný model)', color='gray', linestyle='-.', linewidth=2, alpha=0.7)
    
    # Vykreslení adaptivních a AI filtrů
    ax.plot(frob_errors_mdm[i].numpy(), label='AKF (MDM)', color='green', linewidth=1.5, alpha=0.8)
    ax.plot(frob_errors_mehra[i].numpy(), label='AKF (Mehra)', color='red', linewidth=1.5, alpha=0.8)
    ax.plot(frob_errors_bkn[i].numpy(), label='Bayesian KNet (BKN)', color='blue', linewidth=1.8)
    ax.plot(frob_errors_knetR[i].numpy(), label='KNet with Known R', color='cyan', linestyle='--', linewidth=1.8)
    
    # Benchmarková ideální čára (Chyba = 0)
    ax.axhline(1e-6, color='black', linestyle=':', linewidth=2, label='Optimum (Ideální KF = 0)')
    
    ax.set_title(f"Trajektorie {i+1}: Vývoj chyby Kalmanova zisku při Model Mismatch ($||K - K_{{KF, ideal}}||_F$)")
    ax.set_ylabel("Chyba zisku (Log. měřítko)")
    ax.set_yscale('log')
    ax.grid(True, which="both", ls="--", alpha=0.5)
    
    if i == 0:
        ax.legend(loc='upper right')
        
    num_kf_m_ss.append(frob_errors_kf_m[i][idx_steady:].mean().item())
    num_bkn_ss.append(frob_errors_bkn[i][idx_steady:].mean().item())
    num_knetR_ss.append(frob_errors_knetR[i][idx_steady:].mean().item())
    num_mdm_ss.append(frob_errors_mdm[i][idx_steady:].mean().item())
    num_mehra_ss.append(frob_errors_mehra[i][idx_steady:].mean().item())

axes[-1].set_xlabel("Časový krok [k]")
plt.tight_layout()
plt.show()

# --- Výpis numerických výsledků ---
print("\n" + "="*85)
print("NUMERICKÉ POROVNÁNÍ KALMANOVA ZISKU V USTÁLENÉM STAVU (Model Mismatch)")
print(f"Hodnoceno jako průměrná Frobeniova norma z posledních {steady_state_ratio*100:.0f} % kroků.")
print("CÍL: Číslo co nejblíže k nule (AI by mělo překonat chybný statický KF)")
print("="*85)
print(f"{'Filtr':<35} | {'Průměrná SS Chyba (norma)':<25}")
print("-" * 85)
print(f"{'Statický KF (Nepřesný model)':<35} | {np.mean(num_kf_m_ss):<25.6f} (Baseline)")
print(f"{'AKF (MDM)':<35} | {np.mean(num_mdm_ss):<25.6f}")
print(f"{'AKF (Mehra)':<35} | {np.mean(num_mehra_ss):<25.6f}")
print(f"{'Bayesian KNet (BKN)':<35} | {np.mean(num_bkn_ss):<25.6f}")
print(f"{'KNet with Known R':<35} | {np.mean(num_knetR_ss):<25.6f}")
print("="*85)